## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [13]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [15]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023A7B77FE50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023A7B77F280>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [17]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish! It's great to meet you.  \n\nBeing a Chief AI Engineer is an exciting role. What kind of projects are you working on these days? \n\nI'm eager to learn more about your work and how AI is being applied in your field.\n", response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 22, 'total_tokens': 81, 'completion_time': 0.107272727, 'prompt_time': 0.000134419, 'queue_time': 0.01523972, 'total_time': 0.107407146}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa862d9d-d97d-4903-a435-46bdbe57b2c5-0', usage_metadata={'input_tokens': 22, 'output_tokens': 59, 'total_tokens': 81})

In [18]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat. 😊  \n", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 97, 'total_tokens': 140, 'completion_time': 0.078181818, 'prompt_time': 0.003447128, 'queue_time': 0.009562351, 'total_time': 0.081628946}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-18f3c7ff-5b28-4505-8421-8ca6555e9cd6-0', usage_metadata={'input_tokens': 97, 'output_tokens': 43, 'total_tokens': 140})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [19]:
!pip install langchain_community

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [21]:
config={"configurable":{"session_id":"chat1"}}

In [22]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [23]:
response.content

"That's great to meet you, Krish! \n\nIt's awesome that you're a Chief AI Engineer. That's a really exciting and impactful role. \n\nWhat kind of work are you doing in the field of AI? I'm always eager to learn more about the innovative things people are creating with artificial intelligence.  \n\nPerhaps you could tell me:\n\n* **What industry are you in?**\n* **What are some of the biggest challenges you're facing in your work?**\n* **What are you most excited about for the future of AI?**\n\n\nI'm ready to listen and learn!  \n\n"

In [24]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish, as you told me at the beginning of our conversation. 😊  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 172, 'total_tokens': 204, 'completion_time': 0.058181818, 'prompt_time': 0.005325053, 'queue_time': 0.008145265, 'total_time': 0.063506871}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24efd954-00a1-41c5-9263-4a8abdba0111-0', usage_metadata={'input_tokens': 172, 'output_tokens': 32, 'total_tokens': 204})

In [25]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John!  It's nice to meet you.  \n\nWhat can I do for you today?\n"

In [27]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nIs there anything else I can help you with? 😄\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [28]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [29]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish! It's nice to meet you. \n\nIs there anything I can help you with today?  😊 \n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 31, 'total_tokens': 60, 'completion_time': 0.052727273, 'prompt_time': 0.000341158, 'queue_time': 0.013369232, 'total_time': 0.053068431}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-706f1e91-71f4-4c9c-abc7-f4a2e71f3461-0', usage_metadata={'input_tokens': 31, 'output_tokens': 29, 'total_tokens': 60})

In [30]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [31]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hello Krish! 👋 \n\nIt's nice to meet you. How can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 31, 'total_tokens': 58, 'completion_time': 0.058905572, 'prompt_time': 0.001825684, 'queue_time': 0.012116456000000001, 'total_time': 0.060731256}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2217ef67-a0d1-4c62-b1b0-2250bfa98f55-0', usage_metadata={'input_tokens': 31, 'output_tokens': 27, 'total_tokens': 58})

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish! 😊 I remember that.  \n\nIs there anything else I can help you with?  \n'

In [33]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [34]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 👋  \n\nआप कैसे हैं? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [35]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [36]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! मैं आपकी सहायता करने के लिए तैयार हूँ। आपके कोई सवाल हैं? 😊  \n'

In [37]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [38]:
response.content

'आपका नाम कृष्ण है।  😄  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [39]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\Chatbot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Chatbot\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mohan\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.c

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have taste buds or personal preferences!  \n\nWhat's YOUR favorite ice cream flavor? 😊  \n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2?". \n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIs there anything else I can help you with?\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant with no memory of past conversations, I don't know what math problem you asked. \n\nIf you'd like to ask me a math problem now, I'm happy to help! 😊  \n\n"